# Main Library

In [1]:
# Reading data
import pandas as pd
import numpy as np
import os, nltk

# Data Preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Building Deep Learning Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Input
from tensorflow.keras.models import Model, load_model
import tensorflow.keras as k

# Reading Data

In [2]:
path = r'D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\Data.txt'
data = pd.read_csv(path, sep='\t', names=['english', 'arabic', 'none'])
data = data.drop(columns='none', axis=1)
data = data.iloc[:4000]
data.head()

,english,arabic
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Duck!,اخفض رأسك!
3,Duck!,اخفضي رأسك!
4,Duck!,اخفضوا رؤوسكم!


In [3]:
data.isnull().sum()

english    0
arabic     0
dtype: int64

In [4]:
data.shape

(4000, 2)

# Data Preprocessing

## Ecoder For English Sentence

In [5]:
english_text = data.english.tolist()

token = Tokenizer()
token.fit_on_texts(english_text)
english_seq = token.texts_to_sequences(english_text)

max_eng = max([len(sen) for sen in english_seq])
print('The MAx Length of English Sentence is ---> ', max_eng)

english_pad = pad_sequences(english_seq, padding='post', maxlen=max_eng)
english_pad_array = np.array(english_pad)

print('The Shape of English padding is ---> ', english_pad_array.shape)
english_dict = token.word_index
num_eng_words = len(english_dict) + 1

print("The Number of English Word is --> ",  num_eng_words)

The MAx Length of English Sentence is --->  7
The Shape of English padding is --->  (4000, 7)
The Number of English Word is -->  1643


# Decoding Input Arabic Text

In [6]:
arabic_text = []
for text in data.arabic:
    arabic_text.append('<START>' + text + '<END>')

token = Tokenizer()
token.fit_on_texts(arabic_text)
arabic_seq = token.texts_to_sequences(arabic_text)

max_ar = max([len(sen) for sen in arabic_seq])
print('The MAx Length of English Sentence is ---> ', max_ar)

arabic_pad = pad_sequences(arabic_seq, padding='post', maxlen=max_ar)
arabic_pad_array = np.array(arabic_pad)

print('The Shape of English padding is ---> ', arabic_pad_array.shape)
arabic_dict = token.word_index
num_ar_words = len(arabic_dict) + 1

print("The Number of English Word is --> ",  num_ar_words)

The MAx Length of English Sentence is --->  10
The Shape of English padding is --->  (4000, 10)
The Number of English Word is -->  3996


# Decoding Output Arabic

In [7]:
decoder_output_data = []
for text in arabic_seq:
    decoder_output_data.append(text[1:])
    
decoder_padding = pad_sequences(decoder_output_data, padding='post', maxlen=max_ar)
decoder_output_data = np.array(to_categorical(decoder_padding, num_ar_words))

# Building Encoder Decoder Model

In [8]:
# Building Encoder Layer

encoder_input = Input(shape=(None, ))
encoder_embeding = Embedding(num_eng_words, 256, mask_zero=True)(encoder_input)
encoder_output, state_h, state_c = LSTM(128, return_state=True)(encoder_embeding)
encoder_states = [state_h, state_c]

# Building Decoder Layer
decoder_input = Input(shape=(None, ))
decoder_embeding = Embedding(num_ar_words, 256, mask_zero=True)(decoder_input)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embeding, initial_state=encoder_states)
decoder_dence = Dense(num_ar_words, activation='softmax')
output = decoder_dence(decoder_output)

# Collect Encoder-Decoder
model = Model([encoder_input, decoder_input], output)
model.compile(optimizer=k.optimizers.RMSprop(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    420608      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    1022976     ['input_2[0][0]']                
                                                                                              

In [9]:
model.fit([english_pad_array,arabic_pad_array], decoder_output_data, epochs=200, batch_size=300)

Epoch 1/200
14/14 [==============================] - 11s 53ms/step - loss: 3.4653 - accuracy: 0.2011
Epoch 2/200
14/14 [==============================] - 0s 33ms/step - loss: 2.4882 - accuracy: 0.2372
Epoch 3/200
14/14 [==============================] - 0s 33ms/step - loss: 2.4073 - accuracy: 0.2946
Epoch 4/200
14/14 [==============================] - 0s 33ms/step - loss: 2.3284 - accuracy: 0.3328
Epoch 5/200
14/14 [==============================] - 0s 33ms/step - loss: 2.2539 - accuracy: 0.3721
Epoch 6/200
14/14 [==============================] - 0s 34ms/step - loss: 2.1861 - accuracy: 0.4073
Epoch 7/200
14/14 [==============================] - 0s 33ms/step - loss: 2.1246 - accuracy: 0.4226
Epoch 8/200
14/14 [==============================] - 0s 33ms/step - loss: 2.0723 - accuracy: 0.4255
Epoch 9/200
14/14 [==============================] - 0s 33ms/step - loss: 2.0294 - accuracy: 0.4278
Epoch 10/200
14/14 [==============================] - 0s 33ms/step - loss: 1.9943 - accuracy: 0.429

14/14 [==============================] - 0s 34ms/step - loss: 0.5264 - accuracy: 0.7963
Epoch 83/200
14/14 [==============================] - 0s 33ms/step - loss: 0.5103 - accuracy: 0.8043
Epoch 84/200
14/14 [==============================] - 0s 33ms/step - loss: 0.4999 - accuracy: 0.8095
Epoch 85/200
14/14 [==============================] - 0s 33ms/step - loss: 0.4870 - accuracy: 0.8144
Epoch 86/200
14/14 [==============================] - 0s 34ms/step - loss: 0.4749 - accuracy: 0.8194
Epoch 87/200
14/14 [==============================] - 0s 34ms/step - loss: 0.4609 - accuracy: 0.8246
Epoch 88/200
14/14 [==============================] - 0s 34ms/step - loss: 0.4521 - accuracy: 0.8295
Epoch 89/200
14/14 [==============================] - 0s 33ms/step - loss: 0.4400 - accuracy: 0.8342
Epoch 90/200
14/14 [==============================] - 0s 33ms/step - loss: 0.4287 - accuracy: 0.8401
Epoch 91/200
14/14 [==============================] - 0s 33ms/step - loss: 0.4176 - accuracy: 0.8456
Epo

14/14 [==============================] - 0s 33ms/step - loss: 0.0725 - accuracy: 0.9500
Epoch 163/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0712 - accuracy: 0.9509
Epoch 164/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0698 - accuracy: 0.9507
Epoch 165/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0689 - accuracy: 0.9505
Epoch 166/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0677 - accuracy: 0.9509
Epoch 167/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0665 - accuracy: 0.9504
Epoch 168/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0658 - accuracy: 0.9505
Epoch 169/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0645 - accuracy: 0.9522
Epoch 170/200
14/14 [==============================] - 0s 33ms/step - loss: 0.0636 - accuracy: 0.9505
Epoch 171/200
14/14 [==============================] - 0s 34ms/step - loss: 0.0627 - accuracy: 0

In [14]:
def make_refrece_model():
    encoder_model_refrence = Model(encoder_input, encoder_states)
    
    decoder_state_h = Input(shape=(128, ))
    decoder_state_c = Input(shape=(128, ))
    decoder_inputs_states = [decoder_state_h, decoder_state_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embeding, initial_state=decoder_inputs_states)
    decoder_states = [state_h, state_c]
    
    decoder_outputs = decoder_dence(decoder_outputs)
    decoder_model_refrence = Model([decoder_input] + decoder_inputs_states,
                                   [decoder_outputs] + decoder_states)
    
    return encoder_model_refrence, decoder_model_refrence

In [25]:
def str_to_token(sen):
    words = sen.lower().split()
    token_list = []
    for word in words:
        try:
            token_list.append(english_dict[word])
        except:
            print("THe sentence is not recognize ")
            run()
    return pad_sequences([token_list], maxlen=max_eng, padding="post")

In [26]:
enc_model, dec_model = make_refrece_model()
enc_model.save(r"D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\encoder_model")
enc_model.save(r"D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\decoder_model")

INFO:tensorflow:Assets written to: D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\encoder_model\assets


INFO:tensorflow:Assets written to: D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\encoder_model\assets


INFO:tensorflow:Assets written to: D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\decoder_model\assets


INFO:tensorflow:Assets written to: D:\Courses language programming\LLM - Transformer - NLP\NLP - Complete Course\Data\ara-eng\decoder_model\assets


In [31]:
def run():
    enc_model, dec_model = make_refrece_model()

    for sen in range(english_pad_array.shape[0]):
        states_values = enc_model.predict(str_to_token(input("Enter an English Sentence:  ")))

        empty_target = np.zeros((1, 1))
        empty_target[0, 0] = arabic_dict["start"]
        stopping_condition = False
        decoded_translation = ""
        while not stopping_condition:
            dec_output, h, c = dec_model.predict([empty_target] + states_values)
            sampled_word_index = np.argmax(dec_output[0, -1, :])
            sampled_word = None
            for word, index in arabic_dict.items():
                if sampled_word_index == index:
                    decoded_translation += " " + word
                    sampled_word = word
                if sampled_word == "end" or len(decoded_translation.split()) > max_ar:
                    stopping_condition = True

            empty_target = np.zeros((1, 1))
            empty_target[0, 0] = sampled_word_index
            states_values = [h, c]

        print(decoded_translation[:-3])

In [33]:
run()